<a href="https://colab.research.google.com/github/VenkteshV/Question_duplicate_detection/blob/main/Week8/Comparing_NERs_on_jacc_pairs_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip --quiet install --upgrade git+https://github.com/flairNLP/flair.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from tqdm import tqdm
import spacy
from nltk.corpus import stopwords
import nltk
import re
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
nltk.download('punkt')
# from flair.data import Sentence
# from flair.models import SequenceTagger
nlp = spacy.load("en_core_web_sm")
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("drive/My Drive/Extramarks Project/Dataset versions/Model1-Semantic/Jacc_sim_ignore_syllabus.csv", index_col = 0 )

In [ ]:
df.head()

,syllabus_id1,syllabus_id2,ques1,ques2,question_id1,question_id2
0,700295,700313,how many π bonds are present in ferrocene,how many π are present in ferrocene,2073346,2073271
1,700295,731162,how many π bonds are present in ferrocene,how many π bonds are present in silicon dioxide,2073346,1909343
2,698013,697895,what is covalency of chlorine atom in second e...,what is covalency of chlorine atom in second e...,2073324,2073165
3,697895,697895,calculate value of x y here x and y are the to...,calculate value of x y here x and y are the to...,2073215,2073343
4,697895,698013,in phosphorus acid if x is number of non bondi...,in phosphorus acid if x is number of non bondi...,2073202,2073272


### Using Flair 

In [ ]:
# tagger = SequenceTagger.load('ner-ontonotes')

In [ ]:
# sentence = Sentence('George Washington went to Washington.')
# tagger.predict(sentence)
# for entity in sentence.get_spans('ner'):
#     print(f'entity.text is: "{entity.text}"')
#     print(f'entity.start_position is: "{entity.start_position}"')
#     print(f'entity.end_position is: "{entity.end_position}"')
#     print(f'entity.tag is: "{entity.tag}"')
#     print()

In [ ]:
# df = df[:100]

In [ ]:
# non_dupl = []

# for index, row in df.iterrows():

#   if index % 25 == 0:
#     print(index)

#   ques1 = Sentence(row['ques1'])
#   ques2 = Sentence(row['ques2'])

#   words_1 = {}
#   words_2 = {}

#   tagger.predict(ques1)
#   tagger.predict(ques2)

#   for entity in ques1.get_spans('ner'):
#     words_1[entity.text] = entity.tag
  
#   for entity in ques2.get_spans('ner'):
#     words_2[entity.text] = entity.tag
#     if entity.text not in words_1:
#       non_dupl_row = pd.Series(row)
#       non_dupl_row['diff_entity'] = entity.text # this is partly wrong since multiple entities can differ
#       non_dupl_row['diff_entity_tag'] = entity.tag # this is partly wrong since multiple entities can differ
#       non_dupl.append(non_dupl_row)
#       break

#   for entity in ques1.get_spans('ner'):
#     if entity.text not in words_2:
#       non_dupl_row = pd.Series(row)
#       non_dupl_row['diff_entity'] = entity.text # this is partly wrong since multiple entities can differ
#       non_dupl_row['diff_entity_tag'] = entity.tag # this is partly wrong since multiple entities can differ
#       non_dupl.append(non_dupl_row)
#       break

In [ ]:
# flairDf = pd.DataFrame(non_dupl, columns = ['syllabus_id1','syllabus_id2', 'ques1','ques2','question_id1','question_id2', 'diff_entity', 'diff_entity_tag'])
# flairDf.to_csv("drive/My Drive/Extramarks Project/Dataset versions/Model2-Keyword/flair_ner_recognition.csv")
# flairDf

### Using Spacy

In [ ]:
non_dupl = []
stops = set(['a', 'the', 'an', 's'])
# s added for cases like henry's henry s

for index, row in df.iterrows():
  if index % 25 == 0:
    print(index)

  word_list_1 = nltk.word_tokenize(row['ques1'])
  word_list_1 = [word if word not in stops else '' for word in word_list_1]
  lemmatized_output_1 = ' '.join([lemmatizer.lemmatize(w) for w in word_list_1])
  lemmatized_output_1 = re.sub(' +', ' ', lemmatized_output_1)
  lemmatized_output_1 = re.sub("'s", '', lemmatized_output_1)

  word_list_2 = nltk.word_tokenize(row['ques2'])
  word_list_2 = [word if word not in stops else '' for word in word_list_2]
  lemmatized_output_2 = ' '.join([lemmatizer.lemmatize(w) for w in word_list_2])
  lemmatized_output_2 = re.sub(' +', ' ', lemmatized_output_2)
  lemmatized_output_2 = re.sub("'s", '', lemmatized_output_2)


  ques1 = nlp(lemmatized_output_1)
  ques2 = nlp(lemmatized_output_2)


  words_1 = {}
  words_2 = {}

  for entity in ques1.ents:
    words_1[entity.text] = entity.label_
  
  for entity in ques2.ents:
    words_2[entity.text] = entity.label_
    if entity.text not in words_1:
      non_dupl_row = pd.Series(row)
      non_dupl_row['diff_entity'] = entity.text # this is partly wrong since multiple entities can differ
      non_dupl_row['diff_entity_tag'] = entity.label_ # this is partly wrong since multiple entities can differ
      non_dupl.append(non_dupl_row)
      break

  for entity in ques1.ents:
    if entity.text not in words_2:
      non_dupl_row = pd.Series(row)
      non_dupl_row['diff_entity'] = entity.text # this is partly wrong since multiple entities can differ
      non_dupl_row['diff_entity_tag'] = entity.label_ # this is partly wrong since multiple entities can differ
      non_dupl.append(non_dupl_row)
      break

0
25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500
525
550
575
600
625
650
675
700
725
750
775
800
825
850
875
900
925
950
975
1000
1025
1050
1075
1100
1125
1150
1175
1200
1225
1250
1275
1300
1325
1350
1375
1400
1425
1450
1475
1500
1525
1550
1575
1600
1625
1650
1675
1700
1725
1750
1775
1800
1825
1850
1875
1900
1925
1950
1975
2000
2025
2050
2075
2100
2125
2150
2175
2200
2225
2250
2275
2300
2325
2350
2375
2400
2425
2450
2475
2500
2525
2550
2575
2600
2625
2650
2675
2700
2725
2750
2775
2800
2825
2850
2875
2900
2925
2950
2975
3000
3025
3050
3075
3100
3125
3150
3175
3200
3225
3250
3275
3300
3325
3350
3375
3400
3425
3450
3475
3500
3525
3550
3575
3600
3625
3650
3675
3700
3725
3750
3775
3800
3825
3850
3875
3900
3925
3950
3975
4000
4025
4050
4075
4100
4125
4150
4175
4200
4225
4250
4275
4300
4325
4350
4375
4400
4425
4450
4475
4500
4525
4550
4575
4600
4625
4650
4675
4700
4725
4750
4775
4800
4825
4850
4875
4900
4925
4950
4975
5000
5025
5050
5075
5100
5125
5150
5175
5200


In [ ]:
spacyDf = pd.DataFrame(non_dupl, columns = ['syllabus_id1','syllabus_id2', 'ques1','ques2','question_id1','question_id2', 'diff_entity', 'diff_entity_tag'])
spacyDf.to_csv("drive/My Drive/Extramarks Project/Dataset versions/Model2-Keyword/LEMMA_spacy_ner_recognition.csv")
spacyDf

,syllabus_id1,syllabus_id2,ques1,ques2,question_id1,question_id2,diff_entity,diff_entity_tag
7,697707,730884,under the identical conditions of temperature ...,under the identical conditions of temperature ...,2073015,1809127,two,CARDINAL
13,698013,697955,the incorrect order of solubility in water is ...,the incorrect order of solubility in water is ...,2072803,2071907,mgs2o3,PERSON
35,698013,733603,nitrogen gas is liberated by thermal decomposi...,nitrogen is liberated by the thermal decomposi...,2072114,1887817,nitrogen,ORG
46,700313,700313,what order of cfse is incorrect ni dmg 2 ni e...,what order of cfse is incorrect ni dmg 2 ni e...,2072035,2072968,22,CARDINAL
46,700313,700313,what order of cfse is incorrect ni dmg 2 ni e...,what order of cfse is incorrect ni dmg 2 ni e...,2072035,2072968,2 2,CARDINAL
...,...,...,...,...,...,...,...,...
6585,417499,16141,in a three column cash book,in a triple column cash book,1458494,939616,three,CARDINAL
6586,417499,16141,in a three column cash book,in a triple column cash book,1458494,927020,three,CARDINAL
6590,511235,5936,combine the following sentences into a single ...,combine the following sentences into a single ...,1584771,1464986,ruth,PERSON
6640,511206,4770,tim said that he,he said that,1551779,494476,tim,PERSON


In [ ]:
word_list_1 = nltk.word_tokenize("hinsberg reagent is")
word_list_1 = [word if word not in stops else '' for word in word_list_1]
lemmatized_output_1 = ' '.join([lemmatizer.lemmatize(w) for w in word_list_1])
lemmatized_output_1 = re.sub(' +', ' ', lemmatized_output_1)
lemmatized_output_1 = re.sub("'s", '', lemmatized_output_1)

word_list_2 = nltk.word_tokenize(row['ques2'])
word_list_2 = [word if word not in stops else '' for word in word_list_2]
lemmatized_output_2 = ' '.join([lemmatizer.lemmatize(w) for w in word_list_2])
lemmatized_output_2 = re.sub(' +', ' ', lemmatized_output_2)
lemmatized_output_2 = re.sub("'s", '', lemmatized_output_2)


nlp(lemmatized_output_1).ents.equals(nlp(lemmatized_output_2).entsž

AttributeError: ignored